<a href="https://colab.research.google.com/github/douglasmasho/MedAlgo/blob/main/Survival.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
import os
import glob

# Mount Google Drive
drive.mount('/content/drive')

# data_dir = '/content/drive/MyDrive/BRATS/MICCAI_BraTS_2019_Data_Training'

# survival_data = os.path.join(data_dir, "survival_data.csv")
# hgg_images = os.path.join(data_dir, "HGG")
# lgg_images = os.path.join(data_dir, "LGG")

# print(hgg_images)

Mounted at /content/drive


In [24]:
!pip install monai

In [ ]:
import os
import numpy as np
import pandas as pd
import nibabel as nib
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img
from tensorflow.keras import layers, models
import tensorflow as tf

# Load survival data
survival_data = pd.read_csv('/content/drive/MyDrive/BRATS/MICCAI_BraTS_2019_Data_Training/processed_survival_data.csv')

# Filter subjects with non-NA Age and Survival
survival_data = survival_data.dropna(subset=['Age', 'Survival', 'Status'])

# Process the survival data
def process_survival(survival):
    if 'ALIVE' in survival:
        days = int(survival.split('(')[1].split()[0])
        return days
    else:
        return -int(survival)

survival_data['Processed_Survival'] = survival_data['Survival'].apply(process_survival)

# Define a function to get 2D slices from 3D MRI images
def get_2d_slices(img_3d):
    slices = []
    for i in range(img_3d.shape[-1]):
        slice_2d = img_3d[:, :, i]
        # Add a channel dimension (height, width, channels)
        slice_2d = np.expand_dims(slice_2d, axis=-1)
        slices.append(slice_2d)
    return slices

# Load and preprocess the data
def load_and_preprocess_data(root_dirs, df):
    images = []
    survival_days = []
    statuses = []
    ages = []

    for root_dir in root_dirs:
        for subject_dir in os.listdir(root_dir):
            subject_path = os.path.join(root_dir, subject_dir)
            if os.path.isdir(subject_path):
                img_path = os.path.join(subject_path, f'{subject_dir}_t1ce.nii')
                if os.path.isfile(img_path):
                    # Extract Brats19ID from the subject directory name
                    brats_id = subject_dir
                    if brats_id in df['BraTS19ID'].values:
                        # Get the corresponding survival and age data
                        row = df[df['BraTS19ID'] == brats_id].iloc[0]
                        age = row['Age']
                        survival = row['Processed_Survival']
                        status = row['Status']

                        img_3d = nib.load(img_path).get_fdata()
                        slices = get_2d_slices(img_3d)

                        for slice_2d in slices:
                            # Resize the image if needed
                            img_resized = array_to_img(slice_2d, scale=True).resize((64, 64))
                            img_array = img_to_array(img_resized)
                            images.append(img_array)
                            survival_days.append(survival)
                            statuses.append(1 if status == 'ALIVE' else 0)
                            ages.append(age)

    return np.array(images), np.array(ages), np.array(survival_days), np.array(statuses)

# Define directories for both HGG and LGG
hgg_dir = '/content/drive/MyDrive/BRATS/MICCAI_BraTS_2019_Data_Training/HGG'
lgg_dir = '/content/drive/MyDrive/BRATS/MICCAI_BraTS_2019_Data_Training/LGG'
all_dirs = [hgg_dir, lgg_dir]

# Load and preprocess the data
images, ages, survival_days, statuses = load_and_preprocess_data(all_dirs, survival_data)

# Split the dataset into training and validation sets
X_train, X_val, age_train, age_val, y_train, y_val, status_train, status_val = train_test_split(
    images, ages, survival_days, statuses, test_size=0.2, random_state=42
)

# Define the model
def create_model(input_shape):
    inputs = tf.keras.Input(shape=input_shape)
    x = layers.Conv2D(32, (3, 3), activation='relu')(inputs)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(128, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu')(x)

    age_input = tf.keras.Input(shape=(1,))
    age_dense = layers.Dense(64, activation='relu')(age_input)

    combined = layers.concatenate([x, age_dense])

    # Output layer for survival days
    survival_output = layers.Dense(1, name='survival')(combined)

    # Output layer for status
    status_output = layers.Dense(1, activation='sigmoid', name='status')(combined)

    model = tf.keras.Model(inputs=[inputs, age_input], outputs=[survival_output, status_output])
    model.compile(optimizer='adam',
                  loss={'survival': 'mse', 'status': 'binary_crossentropy'},
                  metrics={'status': 'accuracy'})
    return model

# Create and train the model
input_shape = (64, 64, 1)
model = create_model(input_shape)

history = model.fit(
    [X_train, age_train],
    {'survival': y_train, 'status': status_train},
    epochs=10,
    batch_size=32,
    validation_data=([X_val, age_val], {'survival': y_val, 'status': status_val})
)

# Evaluate the model
loss, survival_loss, status_loss, status_accuracy = model.evaluate([X_val, age_val], {'survival': y_val, 'status': status_val})
print(f"Validation Loss: {loss}")
print(f"Survival Loss: {survival_loss}")
print(f"Status Loss: {status_loss}")
print(f"Status Accuracy: {status_accuracy}")

# Save the model if needed
model.save('glioma_survival_model_with_status.h5')


Epoch 1/10
822/822 ━━━━━━━━━━━━━━━━━━━━ 173s 206ms/step - loss: 153426.2812 - status_accuracy: 0.8827 - val_loss: 132299.5156 - val_status_accuracy: 1.0000
Epoch 2/10
822/822 ━━━━━━━━━━━━━━━━━━━━ 203s 208ms/step - loss: 130522.7812 - status_accuracy: 1.0000 - val_loss: 117046.3750 - val_status_accuracy: 1.0000
Epoch 3/10
822/822 ━━━━━━━━━━━━━━━━━━━━ 211s 219ms/step - loss: 115311.5234 - status_accuracy: 1.0000 - val_loss: 107466.8125 - val_status_accuracy: 1.0000
Epoch 4/10
822/822 ━━━━━━━━━━━━━━━━━━━━ 190s 205ms/step - loss: 100282.5078 - status_accuracy: 1.0000 - val_loss: 94466.0078 - val_status_accuracy: 1.0000
Epoch 5/10
822/822 ━━━━━━━━━━━━━━━━━━━━ 203s 206ms/step - loss: 81526.8984 - status_accuracy: 1.0000 - val_loss: 78304.4609 - val_status_accuracy: 1.0000
Epoch 6/10
822/822 ━━━━━━━━━━━━━━━━━━━━ 201s 205ms/step - loss: 64080.3867 - status_accuracy: 1.0000 - val_loss: 69434.4141 - val_status_accuracy: 1.0000
Epoch 7/10
534/822 ━━━━━━━━━━━━━━━━━━━━ 55s 194ms/step - loss: 52889.

In [12]:
# WORKS

import os
import numpy as np
import pandas as pd
import nibabel as nib
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img
from tensorflow.keras import layers, models
import tensorflow as tf

# Load survival data
survival_data = pd.read_csv('/content/drive/MyDrive/BRATS/MICCAI_BraTS_2019_Data_Training/survival_data.csv')

# Filter subjects with non-NA Age and Survival
survival_data = survival_data.dropna(subset=['Age', 'Survival'])

# Process the survival data
def process_survival(survival):
    if 'ALIVE' in survival:
        days = int(survival.split('(')[1].split()[0])
        return days
    else:
        return -int(survival)

survival_data['Processed_Survival'] = survival_data['Survival'].apply(process_survival)

# Define a function to get 2D slices from 3D MRI images
def get_2d_slices(img_3d):
    slices = []
    for i in range(img_3d.shape[-1]):
        slice_2d = img_3d[:, :, i]
        # Add a channel dimension (height, width, channels)
        slice_2d = np.expand_dims(slice_2d, axis=-1)
        slices.append(slice_2d)
    return slices

# Load and preprocess the data
def load_and_preprocess_data(root_dirs, df):
    images = []
    labels = []
    ages = []

    for root_dir in root_dirs:
        for subject_dir in os.listdir(root_dir):
            subject_path = os.path.join(root_dir, subject_dir)
            if os.path.isdir(subject_path):
                img_path = os.path.join(subject_path, f'{subject_dir}_t1ce.nii')
                if os.path.isfile(img_path):
                    # Extract Brats19ID from the subject directory name
                    brats_id = subject_dir
                    if brats_id in df['BraTS19ID'].values:
                        # Get the corresponding survival and age data
                        row = df[df['BraTS19ID'] == brats_id].iloc[0]
                        age = row['Age']
                        survival = row['Survival']

                        img_3d = nib.load(img_path).get_fdata()
                        slices = get_2d_slices(img_3d)

                        for slice_2d in slices:
                            # Resize the image if needed
                            img_resized = array_to_img(slice_2d, scale=True).resize((64, 64))
                            img_array = img_to_array(img_resized)
                            images.append(img_array)
                            labels.append(survival)
                            ages.append(age)

    return np.array(images), np.array(ages), np.array(labels)

# Define directories for both HGG and LGG
hgg_dir = '/content/drive/MyDrive/BRATS/MICCAI_BraTS_2019_Data_Training/HGG'
lgg_dir = '/content/drive/MyDrive/BRATS/MICCAI_BraTS_2019_Data_Training/LGG'
all_dirs = [hgg_dir, lgg_dir]

# Load and preprocess the data
images, ages, labels = load_and_preprocess_data(all_dirs, survival_data)

# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val, age_train, age_val = train_test_split(images, labels, ages, test_size=0.2, random_state=42)

# Define the model
def create_model(input_shape):
    inputs = tf.keras.Input(shape=input_shape)
    x = layers.Conv2D(32, (3, 3), activation='relu')(inputs)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(128, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu')(x)

    age_input = tf.keras.Input(shape=(1,))
    age_dense = layers.Dense(64, activation='relu')(age_input)

    combined = layers.concatenate([x, age_dense])
    outputs = layers.Dense(1)(combined)

    model = tf.keras.Model(inputs=[inputs, age_input], outputs=outputs)
    model.compile(optimizer='adam', loss='mse')
    return model

# Create and train the model
input_shape = (64, 64, 1)
model = create_model(input_shape)

history = model.fit(
    [X_train, age_train], y_train,
    epochs=10,
    batch_size=32,
    validation_data=([X_val, age_val], y_val)
)

# Evaluate the model
loss = model.evaluate([X_val, age_val], y_val)
print(f"Validation Loss: {loss}")

# Save the model if needed
model.save('glioma_survival_model.h5')


KeyboardInterrupt: 

NameError: name 'load_images' is not defined

In [ ]:
!pip install --upgrade scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 52.3 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.3.2
    Uninstalling scikit-learn-1.3.2:
      Successfully uninstalled scikit-learn-1.3.2


In [ ]:
!pip install torch

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 119)

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 119)

In [ ]:
!pip install monai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 7.0 MB/s eta 0:00:00


In [ ]:
import numpy as np
import tensorflow as tf
import nibabel as nib
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.utils import Sequence

# Function to load NIfTI images
def load_nifti(file_path):
    img = nib.load(file_path)
    return img.get_fdata()

# Data Generator
class NiftiDataGenerator(Sequence):
    def __init__(self, file_paths, labels, ages, batch_size=2, dim=(64, 64, 64), shuffle=True):
        self.file_paths = file_paths
        self.labels = labels
        self.ages = ages
        self.batch_size = batch_size
        self.dim = dim
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.file_paths) / self.batch_size))

    def __getitem__(self, index):
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        file_paths_temp = [self.file_paths[k] for k in indexes]
        X, y, ages = self.__data_generation(file_paths_temp, indexes)
        return [X, np.expand_dims(ages, -1)], y

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.file_paths))
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __data_generation(self, file_paths_temp, indexes):
        X = np.empty((self.batch_size, *self.dim, 1))
        y = np.empty((self.batch_size), dtype=int)
        ages = np.empty((self.batch_size), dtype=float)

        for i, file_path in enumerate(file_paths_temp):
            image = load_nifti(file_path)
            resized_slices = []
            for z in range(image.shape[2]):
                slice_2d = image[:, :, z]
                slice_2d = np.expand_dims(slice_2d, axis=-1)  # Add channel dimension
                resized_slice = tf.image.resize(slice_2d, size=self.dim[:2])
                resized_slices.append(resized_slice.numpy().squeeze(-1))  # Remove channel dimension
            image_resized = np.stack(resized_slices, axis=-1)
            if image_resized.shape[-1] != self.dim[2]:
                image_resized = image_resized[:, :, :self.dim[2]]
            X[i,] = np.expand_dims(image_resized, axis=-1)  # Add channel dimension for the model
            y[i] = self.labels[indexes[i]]
            ages[i] = self.ages[indexes[i]]

        print(f"Batch X shape: {X.shape}")
        print(f"Batch y shape: {y.shape}")
        print(f"Batch ages shape: {ages.shape}")

        return X, y, ages

# Model Definition
def build_model(input_shape):
    model = models.Sequential()
    model.add(layers.Conv3D(32, (3, 3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling3D((2, 2, 2)))
    model.add(layers.Conv3D(64, (3, 3, 3), activation='relu'))
    model.add(layers.MaxPooling3D((2, 2, 2)))
    model.add(layers.Conv3D(128, (3, 3, 3), activation='relu'))
    model.add(layers.MaxPooling3D((2, 2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))  # For binary classification
    model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Dataset Wrapping
def get_output_signature(input_shape):
    return (
        (tf.TensorSpec(shape=(None, *input_shape), dtype=tf.float32), tf.TensorSpec(shape=(None, 1), dtype=tf.float32)),
        tf.TensorSpec(shape=(None,), dtype=tf.float32)
    )

def create_tf_dataset(generator):
    return tf.data.Dataset.from_generator(
        generator,
        output_signature=get_output_signature(generator())
    )

# Training Code
input_shape = (64, 64, 64, 1)
model = build_model(input_shape)

# Updated file paths and labels
file_paths_train = [
    'path_to_HGG_subject1_t1ce.nii',  # Replace with actual file paths
    'path_to_HGG_subject2_t1ce.nii'   # Replace with actual file paths
]
labels_train = [0, 0]  # Replace with actual labels for HGG
ages_train = [40, 50]  # Replace with actual ages

file_paths_test = [
    'path_to_LGG_subject1_t1ce.nii',  # Replace with actual file paths
    'path_to_LGG_subject2_t1ce.nii'   # Replace with actual file paths
]
labels_test = [1, 1]  # Replace with actual labels for LGG
ages_test = [45, 55]  # Replace with actual ages

batch_size = 2
train_generator = NiftiDataGenerator(file_paths_train, labels_train, ages_train, batch_size=batch_size, shuffle=True)
val_generator = NiftiDataGenerator(file_paths_test, labels_test, ages_test, batch_size=batch_size, shuffle=False)

# Wrap generators in tf.data.Dataset
train_dataset = create_tf_dataset(lambda: train_generator)
val_dataset = create_tf_dataset(lambda: val_generator)

# Train the model
model.fit(train_dataset, validation_data=val_dataset, epochs=10)


FileNotFoundError: No such file or no access: 'path_to_HGG_subject2_t1ce.nii'

In [ ]:
# Load survival data
survival_data = pd.read_csv(survival_data_file)

# Display the column names
print(survival_data.columns)


Index(['BraTS19ID', 'Age', 'Survival', 'ResectionStatus'], dtype='object')


In [ ]:
for epoch in range(10):
    model.train()
    running_loss = 0.0
    for batch in train_loader:
        images = batch["image"].cuda()
        labels = batch["label"].cuda()
        ages = batch["age"].cuda()

        optimizer.zero_grad()

        try:
            with autocast('cuda'):
                outputs = model(images, ages)
                loss = criterion(outputs.squeeze(), labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            running_loss += loss.item()

        except Exception as e:
            print(f"Error: {e}")
            continue

    print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_loader)}")

    # Validation loop
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch in val_loader:
            images = batch["image"].cuda()
            labels = batch["label"].cuda()
            ages = batch["age"].cuda()

            try:
                with autocast('cuda'):
                    outputs = model(images, ages)
                    loss = criterion(outputs.squeeze(), labels)
                val_loss += loss.item()

            except Exception as e:
                print(f"Validation Error: {e}")
                continue

    print(f"Epoch {epoch+1}, Validation Loss: {val_loss / len(val_loader)}")

    # Clear memory
    del batch
    torch.cuda.empty_cache()


Epoch 1, Loss: nan
Epoch 1, Validation Loss: nan


KeyboardInterrupt: 

In [ ]:
 import numpy as np
import nibabel as nib
import os
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Concatenate
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Simplified Data Generator
class MRIDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, image_dirs, survival_data, batch_size=16, image_shape=(128, 128, 1), shuffle=True):
        self.image_dirs = image_dirs
        self.survival_data = survival_data
        self.batch_size = batch_size
        self.image_shape = image_shape
        self.shuffle = shuffle
        self.indices = list(range(len(self.image_dirs)))
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.image_dirs) / self.batch_size))

    def __getitem__(self, index):
        batch_indices = self.indices[index*self.batch_size:(index+1)*self.batch_size]
        batch_dirs = [self.image_dirs[i] for i in batch_indices]
        batch_images = []
        batch_survival = []
        batch_age = []

        for img_dir in batch_dirs:
            img, subject_id = self.load_image_from_folder(img_dir)
            if img is not None:
                batch_images.append(img)
                survival, age = self.get_survival_data(subject_id)
                batch_survival.append(survival)
                batch_age.append(age)

        batch_images = np.array(batch_images, dtype=np.float32)
        batch_survival = np.array(batch_survival, dtype=np.float32)
        batch_age = np.array(batch_age, dtype=np.float32)

        # Ensure shapes are correct
        batch_images = np.reshape(batch_images, (batch_images.shape[0], *self.image_shape))
        batch_age = np.reshape(batch_age, (-1, 1))  # Ensure age is 2D

        return {'image_input': batch_images, 'age_input': batch_age}, batch_survival

    def load_image_from_folder(self, folder):
        for file_name in os.listdir(folder):
            if 't1ce.nii' in file_name:
                img_path = os.path.join(folder, file_name)
                img = nib.load(img_path).get_fdata()
                img = np.expand_dims(img, axis=-1)  # Add channel dimension
                img = img / np.max(img)  # Normalize
                img = np.resize(img, self.image_shape)  # Resize to consistent shape
                return img, os.path.basename(folder)
        return None, None

    def get_survival_data(self, subject_id):
        survival = self.survival_data[self.survival_data['BraTS19ID'] == subject_id]['Survival']
        age = self.survival_data[self.survival_data['BraTS19ID'] == subject_id]['Age']
        return survival.values[0] if not survival.empty else np.nan, age.values[0] if not age.empty else 0

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)

# Preprocess survival data
def preprocess_survival(value):
    if 'ALIVE' in str(value):
        parts = str(value).split()
        if len(parts) > 1 and parts[1].isdigit():
            return float(parts[1])
        else:
            return 10000  # or another suitable high value
    elif pd.isna(value) or str(value).strip() == '':
        return np.nan  # or a placeholder like 0
    else:
        try:
            return float(value)
        except ValueError:
            return np.nan  # Handle any unexpected formats

# Load and preprocess survival data
data_dir = '/content/drive/MyDrive/BRATS/MICCAI_BraTS_2019_Data_Training'
survival_data_file = os.path.join(data_dir, 'survival_data.csv')
survival_data = pd.read_csv(survival_data_file)

# Preprocess survival data
survival_data['Survival'] = survival_data['Survival'].apply(preprocess_survival)
survival_data['Survival'].fillna(survival_data['Survival'].median(), inplace=True)
survival_data['Age'] = StandardScaler().fit_transform(survival_data[['Age']])

# Define directories
hgg_images_dir = os.path.join(data_dir, 'HGG')
lgg_images_dir = os.path.join(data_dir, 'LGG')

# Prepare data
hgg_dirs = [os.path.join(hgg_images_dir, d) for d in os.listdir(hgg_images_dir)]
lgg_dirs = [os.path.join(lgg_images_dir, d) for d in os.listdir(lgg_images_dir)]
all_dirs = hgg_dirs + lgg_dirs

# Split data
train_dirs, test_dirs = train_test_split(all_dirs, test_size=0.2, random_state=42)

# Create data generators
train_generator = MRIDataGenerator(train_dirs, survival_data, batch_size=16)
test_generator = MRIDataGenerator(test_dirs, survival_data, batch_size=16)

# Define a simpler model for debugging
image_input = Input(shape=(128, 128, 1), name='image_input')
age_input = Input(shape=(1,), name='age_input')

# Define CNN part for image processing
x = Conv2D(32, (3, 3), activation='relu')(image_input)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(128, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)

# Concatenate image features with age
x = Concatenate()([x, age_input])
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(1)(x)  # Output layer for regression

# Define the model
model = Model(inputs=[image_input, age_input], outputs=output)
model.compile(optimizer=Adam(), loss='mean_squared_error', metrics=['mae'])

# Print model summary
model.summary()

# Train model
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=test_generator
)

# Evaluate model
loss, mae = model.evaluate(test_generator)
print(f"Mean Absolute Error on test data: {mae}")

# Prediction function
def predict_survival(image, age):
    image = np.expand_dims(image, axis=0)
    age = np.expand_dims(age, axis=0)
    survival = model.predict({'image_input': image, 'age_input': age})
    return survival

# Example prediction
example_batch = next(iter(test_generator))
example_image = example_batch[0]['image_input'][0]
predicted_survival = predict_survival(example_image, np.array([30]))  # Assuming age 30
print(f"Predicted survival: {predicted_survival}")


Model: "functional_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ image_input (InputLayer)  │ (None, 128, 128, 1)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_39 (Conv2D)        │ (None, 126, 126, 32)   │            320 │ image_input[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_39          │ (None, 63, 63, 32)     │              0 │ conv2d_39[0][0]        │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_40 (Conv2D)        │ (None, 61, 61, 64)     │         18,496 │ max_pooling2d_39[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_40          │ (None, 30, 30, 64)     │              0 │ conv2d_40[0][0]        │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_41 (Conv2D)        │ (None, 28, 28, 128)    │         73,856 │ max_pooling2d_40[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_41          │ (None, 14, 14, 128)    │              0 │ conv2d_41[0][0]        │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_13 (Flatten)      │ (None, 25088)          │              0 │ max_pooling2d_41[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ age_input (InputLayer)    │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_11            │ (None, 25089)          │              0 │ flatten_13[0][0],      │
│ (Concatenate)             │                        │                │ age_input[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_26 (Dense)          │ (None, 128)            │      3,211,520 │ concatenate_11[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_13 (Dropout)      │ (None, 128)            │              0 │ dense_26[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_27 (Dense)          │ (None, 1)              │            129 │ dropout_13[0][0]       │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 3,304,321 (12.60 MB)

 Trainable params: 3,304,321 (12.60 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20


ValueError: Exception encountered when calling Functional.call().

[1mInvalid input shape for input Tensor("data_1:0", shape=(None, 1), dtype=float32). Expected shape (None, 128, 128, 1), but input has incompatible shape (None, 1)[0m

Arguments received by Functional.call():
  • inputs={'image_input': 'tf.Tensor(shape=(None, 128, 128, 1), dtype=float32)', 'age_input': 'tf.Tensor(shape=(None, 1), dtype=float32)'}
  • training=True
  • mask={'image_input': 'None', 'age_input': 'None'}

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Define a minimal model
def create_model():
    image_input = Input(shape=(128, 128, 1), name='image_input')
    age_input = Input(shape=(1,), name='age_input')

    x = Conv2D(32, (3, 3), activation='relu')(image_input)
    x = MaxPooling2D((2, 2))(x)
    x = Flatten()(x)

    # Combine features
    combined = Concatenate()([x, age_input])
    x = Dense(64, activation='relu')(combined)
    x = Dropout(0.5)(x)
    output = Dense(1)(x)

    model = Model(inputs=[image_input, age_input], outputs=output)
    model.compile(optimizer=Adam(), loss='mean_squared_error', metrics=['mae'])

    return model

# Dummy data generator to tf.data.Dataset
def create_dataset(batch_size=16):
    num_samples = 100
    image_data = np.random.rand(num_samples, 128, 128, 1).astype(np.float32)
    age_data = np.random.rand(num_samples, 1).astype(np.float32)
    survival_data = np.random.rand(num_samples).astype(np.float32)

    dataset = tf.data.Dataset.from_tensor_slices(((image_data, age_data), survival_data))
    dataset = dataset.batch(batch_size)
    return dataset

# Create and compile model
model = create_model()
model.summary()

# Create dataset
train_dataset = create_dataset()

# Train the model
try:
    history = model.fit(
        train_dataset,
        epochs=2,
        steps_per_epoch=10
    )
except Exception as e:
    print("Error during training:", e)


Model: "functional_20"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ image_input (InputLayer)  │ (None, 128, 128, 1)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_54 (Conv2D)        │ (None, 126, 126, 32)   │            320 │ image_input[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_54          │ (None, 63, 63, 32)     │              0 │ conv2d_54[0][0]        │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_20 (Flatten)      │ (None, 127008)         │              0 │ max_pooling2d_54[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ age_input (InputLayer)    │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_18            │ (None, 127009)         │              0 │ flatten_20[0][0],      │
│ (Concatenate)             │                        │                │ age_input[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_40 (Dense)          │ (None, 64)             │      8,128,640 │ concatenate_18[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_20 (Dropout)      │ (None, 64)             │              0 │ dense_40[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_41 (Dense)          │ (None, 1)              │             65 │ dropout_20[0][0]       │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 8,129,025 (31.01 MB)

 Trainable params: 8,129,025 (31.01 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/2
10/10 ━━━━━━━━━━━━━━━━━━━━ 9s 139ms/step - loss: 398.3152 - mae: 10.7971
Epoch 2/2
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.3174 - mae: 0.9568 


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)
